In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def dataimputer(df,a,b):
  from sklearn.impute import SimpleImputer, KNNImputer
  impute = SimpleImputer(strategy = "most_frequent")
  knnimputer = KNNImputer(n_neighbors=25)
  if len(a)!=0:
    df2 = df.iloc[:,a]
    df2 = knnimputer.fit_transform(df2)
    df2 = pd.DataFrame(df2)
  if len(b)!=0:
    df3 = df.iloc[:,b]
    df3 = impute.fit_transform(df3)
    df3 = pd.DataFrame(df3)
  i=0
  for cc in a:
    df.iloc[:,cc] = df2.iloc[:,i]
    i=i+1
  j=0
  for bb in b:
    df.iloc[:,bb] = df3.iloc[:,j]
    j=j+1
  return df

In [ ]:
def nullfinder(df):
  num = []
  obj = []
  li1 = df.isnull().sum().values
  li2 = df.dtypes.values
  for a in range(0,len(df.columns)):
    if li1[a]>0:
      if li2[a]=="int" or li2[a]=="float":
        num.append(a)
      elif li2[a]=="O":
        obj.append(a)
  return num,obj

In [ ]:
def normalize(df,b,a):
  from sklearn.preprocessing import OneHotEncoder
  from sklearn.preprocessing import StandardScaler
  ohe = OneHotEncoder()
  sc = StandardScaler()

  df2 = df.iloc[:,a]
  df2 = ohe.fit_transform(df2)
  df2 = pd.DataFrame(df2.toarray())

  df3 = df.iloc[:,b]
  df3 = sc.fit_transform(df3)
  df3 = pd.DataFrame(df3)

  df = pd.concat([df2,df3],axis=1)
  return df

In [ ]:
def typefinder(df):
  num2 = []
  obj2 = []
  li2 = df.dtypes.values
  for a in range(0,len(df.columns)):
    if li2[a]=="int" or li2[a]=="float":
      num2.append(a)
    elif li2[a]=="O":
      obj2.append(a)
  return num2,obj2

In [ ]:
train = pd.read_csv("/content/train (1).csv")
train.shape

(1460, 81)

In [ ]:
test = pd.read_csv("/content/test (1).csv")
test.shape

(1459, 80)

In [ ]:
y = train.iloc[:,-1]
train.drop(train.columns[-1],axis=1,inplace=True)

In [ ]:
final = pd.concat([train,test])
val = final.isnull().sum().values

In [ ]:
final = pd.concat([train,test])
num,obj = nullfinder(final)
_f = dataimputer(final,num,obj)
num2,obj2 = typefinder(_f)
_f2 = normalize(_f,num2,obj2)

In [ ]:
train = _f2.iloc[:1460]
test = _f2.iloc[1460:]

In [ ]:
num,obj = nullfinder(train)
train_f = dataimputer(train,num,obj)

In [ ]:
num2,obj2 = typefinder(train_f)
train_f2 = normalize(train_f,num2,obj2)

In [ ]:
num3,obj3 = nullfinder(test)
test_f = dataimputer(test,num3,obj3)

In [ ]:
num4,obj4 = typefinder(test_f)
test_f2 = normalize(test_f,num4,obj4)

In [ ]:
def regressor1(x_train,x_test,y_train):
  from sklearn.linear_model import LinearRegression
  from sklearn.tree import DecisionTreeRegressor
  from sklearn.ensemble import RandomForestRegressor  
  from sklearn.svm import SVR

  lr = LinearRegression()
  lr.fit(x_train,y_train)
  y_pred = lr.predict(x_test)
  
  dtr = DecisionTreeRegressor(max_leaf_nodes=15)
  dtr.fit(x_train,y_train)
  y_pred2 = dtr.predict(x_test)

  rfr = RandomForestRegressor(n_estimators=1500,criterion="absolute_error")
  rfr.fit(x_train,y_train)
  y_pred3 = rfr.predict(x_test)

  svr = SVR()
  svr.fit(x_train,y_train)
  y_pred4 = svr.predict(x_test)

  return y_pred,y_pred2,y_pred3,y_pred4

In [ ]:
train.shape,test.shape

((1460, 289), (1459, 289))

In [ ]:
p1,p2,p3,p4 = regressor1(train,test,y)

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(p1,p2),mean_absolute_error(p2,p3),mean_absolute_error(p3,p4),mean_absolute_error(p4,p1),mean_absolute_error(p4,p2),mean_absolute_error(p1,p3)

(32096.34637715401,
 18322.29446077968,
 41474.382501478634,
 49759.20989659594,
 45090.086680772445,
 21588.967919122686)

In [ ]:
sub = pd.read_csv("/content/sample_submission.csv")

In [ ]:
te = pd.read_csv("/content/test (1).csv")
id = te.iloc[:,0]

In [ ]:
submission = pd.DataFrame({"Id":id,"SalePrice":p3})

In [ ]:
submission.to_csv("Submission.csv",index=False)

In [ ]:
submission=pd.read_csv("Submission.csv")
submission

,Id,SalePrice
0,1461,135634.00
1,1462,164138.32
2,1463,184733.02
3,1464,172326.20
4,1465,216519.72
...,...,...
1454,2915,136508.00
1455,2916,107599.30
1456,2917,155604.87
1457,2918,132867.89


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
model = Sequential()
model.add(Dense(512, kernel_initializer='normal',activation='relu',input_shape=(289,)))
model.add(Dense(512, kernel_initializer='normal',activation='relu'))
model.add(Dense(256, kernel_initializer='normal',activation='relu'))
model.add(Dense(256, kernel_initializer='normal',activation='relu'))
model.add(Dense(128, kernel_initializer='normal',activation='relu'))
model.add(Dense(128, kernel_initializer='normal',activation='relu'))
model.add(Dense(1))

model.summary()
model.compile(loss='mse', optimizer=Adam(lr=0.0001), metrics=['mse','mae'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 512)               148480    
                                                                 
 dense_8 (Dense)             (None, 512)               262656    
                                                                 
 dense_9 (Dense)             (None, 256)               131328    
                                                                 
 dense_10 (Dense)            (None, 256)               65792     
                                                                 
 dense_11 (Dense)            (None, 128)               32896     
                                                                 
 dense_12 (Dense)            (None, 128)               16512     
                                                                 
 dense_13 (Dense)            (None, 1)                

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.fit(train,y,epochs=500)

Epoch 1/500
46/46 [==============================] - 1s 12ms/step - loss: 1042784.1875 - mse: 1042784.1875 - mae: 683.4798
Epoch 2/500
46/46 [==============================] - 1s 13ms/step - loss: 1073960.2500 - mse: 1073960.2500 - mae: 714.0529
Epoch 3/500
46/46 [==============================] - 1s 11ms/step - loss: 874448.0625 - mse: 874448.0625 - mae: 609.0703
Epoch 4/500
46/46 [==============================] - 1s 14ms/step - loss: 989961.9375 - mse: 989961.9375 - mae: 652.1925
Epoch 5/500
46/46 [==============================] - 1s 12ms/step - loss: 1163404.8750 - mse: 1163404.8750 - mae: 719.4120
Epoch 6/500
46/46 [==============================] - 0s 10ms/step - loss: 1339339.0000 - mse: 1339339.0000 - mae: 817.3724
Epoch 7/500
46/46 [==============================] - 1s 13ms/step - loss: 1118261.6250 - mse: 1118261.6250 - mae: 740.3006
Epoch 8/500
46/46 [==============================] - 1s 12ms/step - loss: 963469.6875 - mse: 963469.6875 - mae: 677.4423
Epoch 9/500
46/46 [===

In [ ]:
pr = model.predict(test)

In [ ]:
pr = [a[0] for a in pr]
t2 = pd.read_csv("/content/test (1).csv")
id = t2.iloc[:,0]
submission = pd.DataFrame({"Id":id,"SalePrice":pr})
submission.to_csv("Submission.csv",index=False)
pd.read_csv("Submission.csv")

,Id,SalePrice
0,1461,158149.11
1,1462,225586.40
2,1463,204379.78
3,1464,184544.03
4,1465,222149.25
...,...,...
1454,2915,113594.91
1455,2916,90831.31
1456,2917,167368.40
1457,2918,127618.50
